In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/Priyanka Konda/Desktop/netflix-movie-recommendation/data/netflix_titles.csv/netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
df.shape

(8807, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [6]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [7]:
df = df[df["type"] == "Movie"]

In [8]:
df = df[["title", "listed_in", "cast", "description"]]

In [9]:
df["listed_in"] = df["listed_in"].fillna("")
df["cast"] = df["cast"].fillna("")
df["description"] =df["description"].fillna("")

In [10]:
df.shape

(6131, 4)

In [11]:
df["combined_features"] = (
    df["listed_in"] + " " +
    df["cast"] + " " +
    df["description"]
)

In [12]:
df["combined_features"].head()

0     Documentaries  As her father nears the end of ...
6     Children & Family Movies Vanessa Hudgens, Kimi...
7     Dramas, Independent Movies, International Movi...
9     Comedies, Dramas Melissa McCarthy, Chris O'Dow...
12    Dramas, International Movies Luna Wedler, Jann...
Name: combined_features, dtype: object

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

In [15]:
tfidf_matrix = tfidf.fit_transform(df["combined_features"])

In [16]:
tfidf_matrix.shape

(6131, 5000)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [19]:
df.shape

(6131, 5)

In [20]:
tfidf_matrix.shape

(6131, 5000)

In [21]:
indices = pd.Series(df.index, index=df["title"]).drop_duplicates()

In [22]:
def recommend(movie_title, top_n=10):
    if movie_title not in indices:
        return "Movie not found in the database."

    idx = indices[movie_title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]

    movie_indices = [i[0] for i in similarity_scores]
    return df["title"].iloc[movie_indices]

In [23]:
recommend("Inception")

2372                      One Take
7284    Legendary Weapons of China
870     Amy Tan: Unintended Memoir
1215               Romantik Komedi
3670         Cities of Last Things
3155              The 9th Precinct
7146                        Jugaad
2533                 Fan of Amoory
641               Sisters on Track
4475                      Shirkers
Name: title, dtype: object

In [24]:
indices = pd.Series(df.index, index=df["title"].str.lower()).drop_duplicates()

In [25]:
def recommend(movie_title, top_n=10):
    movie_title = movie_title.lower()

    if movie_title not in indices:
        return "Movie not found in the database."

    idx = indices[movie_title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]

    movie_indices = [i[0] for i in similarity_scores]
    return df["title"].iloc[movie_indices]

In [26]:
pip install fuzzywuzzy python-Levenshtein

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\Priyanka Konda\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
from fuzzywuzzy import process

In [28]:
def recommend(movie_title, top_n=10):
    movie_title = movie_title.lower()
    all_titles = df["title"].str.lower().tolist()

    closest_match = process.extractOne(movie_title, all_titles)
    if closest_match[1] < 60:
        return "Movie not found in the database."

    matched_title = closest_match[0]
    idx = indices[matched_title]

    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]

    movie_indices = [i[0] for i in similarity_scores]
    return df["title"].iloc[movie_indices]


In [29]:
recommend("incepcion")  # typo on purpose

8750                            Winter's Bone
496                            Brick Mansions
866                          Small Town Crime
7237                                  Krystal
2823                    Miracle in Cell No. 7
134                  Clear and Present Danger
1633                                  The 2nd
6337                                     Blow
6956                       Hell or High Water
5218    Belief: The Possession of Janet Moses
Name: title, dtype: object